# Set-up

In [75]:
# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import eugene as eu

# Load and preprocess dataset

# SeqData

In [10]:
sdata_train = eu.datasets.deAlmeida22("train")

Dataset deAlmeida22 Sequences_Train.fa has already been downloaded.
Dataset deAlmeida22 Sequences_activity_Train.txt has already been downloaded.


In [40]:
from eugene.preprocess._preprocessing import downsample_sdata

In [42]:
sdata_train = downsample_sdata(sdata_train, frac=0.1)

In [49]:
sdata_train

SeqData object with = 40229 seqs
seqs = (40229,)
names = (40229,)
rev_seqs = None
ohe_seqs = (40229, 4, 249)
ohe_rev_seqs = None
seqs_annot: 'Dev_log2_enrichment', 'Hk_log2_enrichment', 'Dev_log2_enrichment_scaled', 'Hk_log2_enrichment_scaled', 'Dev_log2_enrichment_quantile_normalized', 'Hk_log2_enrichment_quantile_normalized'
pos_annot: None
seqsm: None
uns: None

In [43]:
eu.pp.ohe_seqs_sdata(sdata_train)

In [45]:
sdata_train.ohe_seqs.shape

(40229, 4, 249)

In [50]:
sdataset_train = sdata_train.to_dataset(target_keys=['Dev_log2_enrichment_scaled', 'Hk_log2_enrichment_scaled'])

No transforms given, assuming just need to tensorize.


In [51]:
sdataset_train[0]

(tensor([ 99., 104., 114.,  51.,  76.,  72., 101., 116.,  95.,  49.,  49.,  52.,
          56.,  55.,  57.,  57.,  95.,  49.,  49.,  52.,  57.,  48.,  52.,  55.,
          95.,  45.,  95., 112., 101.,  97., 107.,  95.,  56.,  52.,  57.,  98.,
         112.,  95., 114., 101., 103., 105., 111., 110.]),
 tensor([[0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
          0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
          0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
          1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
          1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
          0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
          0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 1., 0., 

In [52]:
sdataloader_train = sdataset_train.to_dataloader(batch_size=256, shuffle=True)

In [53]:
sdataloader_train

In [11]:
sdata_val = eu.datasets.deAlmeida22("val")

Dataset deAlmeida22 Sequences_Val.fa has already been downloaded.
Dataset deAlmeida22 Sequences_activity_Val.txt has already been downloaded.


In [ ]:
eu.pp.ohe_seqs_sdata(sdata_val)

In [54]:
sdataset_val = sdata_val.to_dataset(target_keys=['Dev_log2_enrichment_scaled', 'Hk_log2_enrichment_scaled'])

No transforms given, assuming just need to tensorize.


In [55]:
sdataloader_val = sdataset_val.to_dataloader(batch_size=256, shuffle=False)

In [57]:
len(sdataloader_train), len(sdataloader_val)

(158, 159)

# From H5 file

In [80]:
from eugene.dataload.datamodules._H5DataModule import H5DataModule

In [81]:
!wget https://zenodo.org/record/7265991/files/DeepSTARR_data.h5

--2022-12-15 14:36:00--  https://zenodo.org/record/7265991/files/DeepSTARR_data.h5
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 294581428 (281M) [application/octet-stream]
Saving to: ‘DeepSTARR_data.h5’

DeepSTARR_data.h5    24%[===>                ]  68.54M  1.64MB/s    eta 2m 27s 

In [ ]:
filepath = "DeepSTARR_data.h5"
data_module = H5DataModule(filepath, batch_size=100, lower_case=False, transpose=False)

# Instantiate model

In [69]:
from eugene.models._model_zoo import DeepSTARR

In [70]:
model = DeepSTARR(input_len=249, output_dim=2)

In [71]:
model.optimizer

torch.optim.adam.Adam

In [72]:
model.loss_fxn

<function torch.nn.functional.mse_loss(input: torch.Tensor, target: torch.Tensor, size_average: Union[bool, NoneType] = None, reduce: Union[bool, NoneType] = None, reduction: str = 'mean') -> torch.Tensor>

# Train model

In [73]:
eu.train.fit(
    model,
    train_dataloader=sdataloader_train,
    val_dataloader=sdataloader_val
)

Global seed set to 13
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name         | Type        | Params
---------------------------------------------
0 | metric       | R2Score     | 0     
1 | conv1d_block | Conv1DBlock | 92.2 K
2 | dense_block  | DenseBlock  | 7.6 M 
---------------------------------------------
7.7 M     Trainable params
0         Non-trainable params
7.7 M     Total params
30.745    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

torch.Size([256, 120, 245])
tensor(2.5674)


Global seed set to 13


torch.Size([256, 120, 245])
tensor(2.7497)


Training: 0it [00:00, ?it/s]

torch.Size([256, 120, 245])
tensor(1.4132, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.4451, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.5517, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.2892, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.4726, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.2257, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.3507, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.1427, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.2088, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.1111, grad_fn=<MseLossBackward0>)
torch.Size([256, 120, 245])
tensor(1.2722, grad_fn=<MseLossBackward0>)


/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


---

# Scratch